Primeira parte:
    • Representação (genótipo): string de bits #OK
    • Recombinação: “cut-and-crossfill” crossover #OK
    • Probabilidade de Recombinação: 90%
    • Mutação: troca de genes #OK
    • Probabilidade de Mutação: 40%
    • Seleção de pais: ranking - Melhor de 2 de 5 escolhidos aleatoriamente #OK
    • Seleção de sobreviventes: substituição do pior
    • Tamanho da população: 100 #OK
    • Número de filhos gerados: 2
    • Inicialização: aleatória #OK
    • Condição de término: Encontrar a solução, ou 10.000 avaliações de fitness #OK
    • Fitness #OK

In [1]:
from random import randint
import numpy as np

In [2]:
populationSize = 100

### Funções auxiliares

In [3]:
def binaryToDecimal(binary): 
    binary1 = binary
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return decimal

In [4]:
def sortPopulation(population):
    # Ordena uma população pelo seu valor de fitness
    population.sort(key=lambda x:x['fitnessValue'])
    return population

### Criar população

In [5]:
def generateChess():
    aux = []
    while(len(aux) != 8):
        value = generateQueen()
        if not value in aux:
            aux.append(value)
    return aux

In [6]:
def generateQueen():
    queenPosition = ''
    for bitPosition in range(1,4):
        queenPosition += str(randint(0,1))
    return queenPosition

In [7]:
def generatePopulation():
    aux = []
    for populationMember in range(populationSize):
        aux.append({'individual': generateChess() , 'fitnessValue': -1 }) 
        #Cria um objeto composto do individuo e seu valor de fitness    
    return aux

### Quantidade de 'choques'

In [8]:
# Se duas rainhas estiverem na mesma coluna
def columnThreaten(individual):
    threatenColumn = 0
    for column in range(0,8):
        binaryColumn = np.binary_repr(column, width=3)
        if individual.count(binaryColumn) >= 2:
            threatenColumn += 1
    return threatenColumn

In [9]:
# Se duas rainhas estiverem na mesma diagonal
def diagonalThreaten(individual):
    deltaRow = 0
    deltaCol = 0
    threatenDiagonal = 0
    for x in range(0,8):
        for y in range(0,8):
            if x != y:
                deltaRow = abs(x-y)
                deltaCol = abs(binaryToDecimal(int(individual[x])) - binaryToDecimal(int(individual[y])))
                if (deltaRow == deltaCol):
                    threatenDiagonal += 1
    return threatenDiagonal/4

### Calculo do fitness

In [10]:
def fitnessIndividual(individual):
    value = 1 / (1 +(columnThreaten(individual) + diagonalThreaten(individual)))
    return value

In [11]:
def fitness(population):
    fitnessValue = []
    for individual in population:
        individual['fitnessValue'] = fitnessIndividual(individual['individual'])

#### O valor do fitness será dado pela função 1/1+qntdChoques(tabuleiro)
#### Queremos aumentar esse valor de fitness

### Mutação

Escolher um individuo(tabuleiro) aleatório e trocar a posição de uma rainha

In [12]:
def mutationSwap(individual):
    randomQueen = randint(0,8)
    individual['individual'][randomQueen] = generateQueen()
    return individual

### Seleção de pais

In [13]:
def chooseRandomIndividuals(population):
    aux = []
    for i in range(0,5):
        value = randint(0,populationSize)
        aux.append(population[value])
    return aux

In [14]:
chooseRandomIndividuals(generatePopulation())

[{'individual': ['011', '001', '101', '111', '100', '010', '000', '110'],
  'fitnessValue': -1},
 {'individual': ['101', '000', '011', '001', '110', '111', '100', '010'],
  'fitnessValue': -1},
 {'individual': ['110', '010', '101', '111', '001', '100', '000', '011'],
  'fitnessValue': -1},
 {'individual': ['101', '010', '001', '111', '011', '000', '100', '110'],
  'fitnessValue': -1},
 {'individual': ['010', '001', '110', '011', '100', '111', '000', '101'],
  'fitnessValue': -1}]

In [15]:
def selectParents(population):
    parents = chooseRandomIndividuals(population) #Array de objetos onde os elementos são os individuos e o valor do fitness
    parents = sortPopulation(parents) # Ordenando os pais de acordo com o fitnessValue
    return (parents[-2], parents[-1]) #Retornando os dois ultimos elementos da lista que são os com melhores fitness

### Seleção dos sobreviventes

#### Remover os dois menos adaptados da população

In [16]:
def replaceElements(population, filho1, filho2):
    sortPopulation(population)
    population[0] = filho1
    population[1] = filho2

### Recombinação: “cut-and-crossfill”

In [17]:
def copyFirstParent(parent, point):
    son = []
    for x in range(0,point+1): # Se o primeiro pai for escolhido o filho recebera primeiro os genes dele
            son.append(parent['individual'][x])
    return son

In [18]:
def generateParentTwoAux(parent, point):
    aux = []
    for x in range(point+1, 8):
        aux.append(parent['individual'][x])
    for x in range(0, point+1):
        aux.append(parent['individual'][x])
    return aux

In [19]:
def insertValuesSecondParent(parent, point, son):
    parentCircular = generateParentTwoAux(parent, point)
    while(len(son) != 8):
            for x in parentCircular:
                if not x in son: # Se o valor do 2 pai ainda não existe no filho insere
                    son.append(x)
    return {'individual': son, 'fitnessValue': -1}
        

In [20]:
def recombination(parent_one, parent_two):
    son = []
    #point = randint(0,7) # Escolher uma posição para executar
    point = 2
    #chooseFirst = randint(0,1)
    chooseFirst = 0
    if (chooseFirst == 0):
        son = copyFirstParent(parent_one, point) # Se o primeiro pai for escolhido o filho recebera primeiro os genes dele
        return insertValuesSecondParent(parent_two, point, son)
    else:
        son = copyFirstParent(parent_two, point) # Se o segundo pai for escolhido o filho recebera primeiro os genes dele
        return insertValuesSecondParent(parent_one, point, son)

### Procurando pelo melhor fitness

In [21]:
def lookingForFitnessMax(population):
    for individual in population:
        if individual['fitnessValue'] >= 1:
            return True, individual
        else:
            return False, -1

In [26]:
def main():
    iteracao = 0
    population = generatePopulation()
    fitness(population)
    while (iteracao < 10000 | lookingForFitnessMax(population)[0] ):
        print('iteracao: ', iteracao)
        pai1, pai2 = selectParents(population)
        filho1 = mutationSwap(recombination(pai1,pai2))
        filho2 = mutationSwap(recombination(pai1,pai2))
        replaceElements(population, filho1, filho2)
        iteracao += 1
    return lookingForFitnessMax(population)

In [27]:
main()

iteracao:  0
iteracao:  1
iteracao:  2
iteracao:  3
iteracao:  4
iteracao:  5
iteracao:  6
iteracao:  7
iteracao:  8
iteracao:  9


IndexError: list assignment index out of range

In [ ]:
False | False